In [ ]:
import onnxruntime as ort
import numpy as np
import cv2

# Load the ONNX model
model_path = "DenseNet121.onnx"
session = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])

# Preprocess the frame
def preprocess_frame(frame, input_size=(128, 128)):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, input_size)
    frame = frame.astype(np.float32) / 255.0  # Normalize to [0,1]
    frame = np.expand_dims(frame, axis=0)  # Ensure correct input shape (batch_size, height, width, channels)
    return frame

# Perform inference on a single frame
def predict(frame):
    image = preprocess_frame(frame)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    outputs = session.run([output_name], {input_name: image})
    prob = outputs[0][0][0]  # Assuming model outputs probability of "No Fire"
    fire_prob = 1 - prob  # Convert to Fire probability
    return fire_prob

# Detect fire areas
def detect_fire_areas(frame, min_box_size=500):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 + mask2

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_box_size]
    return filtered_contours, mask

# Process live camera feed
def process_live_camera(display_frames=True, min_box_size=500):
    cap = cv2.VideoCapture(0, apiPreference=cv2.CAP_AVFOUNDATION)  # Use webcam
    
    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        fire_contours, fire_mask = detect_fire_areas(frame, min_box_size)
        fire_prob = predict(frame)  # Overall frame fire probability
        text = f"Fire Probability: {fire_prob:.2f}"
        text_color = (0, 0, 255) if fire_prob > 0.5 else (0, 255, 0)  # Red if fire is likely

        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, text, (20, 50), font, 0.8, text_color, 2)

        for contour in fire_contours:
            x, y, w, h = cv2.boundingRect(contour)
            fire_region = fire_mask[y:y+h, x:x+w]
            fire_intensity = np.mean(fire_region) / 255.0  # Normalize between 0-1
            box_color = (0, 0, 255) if fire_intensity > 0.5 else (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), box_color, 2)

        if display_frames:
            cv2.imshow("Live Fire Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

process_live_camera()

: 